In [1]:
import os
import sys
import tensorflow as tf
import numpy as np
import time

In [2]:
# Ensure numpy and tensorflow are working
print(sys.executable)
print(np.__version__)
print(tf.__version__)

# Ensure graphics cards and CUDA drivers are available
print("CUDA available:", tf.test.is_built_with_cuda())

# List all GPUS and then available devices
print("GPU devices:", tf.config.list_physical_devices('GPU'))
print("All devices:", tf.config.list_logical_devices())

c:\Users\arsam\conda\envs\OCR-env\python.exe
1.23.5
2.10.1
CUDA available: True
GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
All devices: [LogicalDevice(name='/device:CPU:0', device_type='CPU'), LogicalDevice(name='/device:GPU:0', device_type='GPU')]


In [7]:
# Force GPU usage
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    # Simple matrix multiplication test
    with tf.device('/GPU:0'):
        a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]) #2x3 matrix
        b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]) #3x2 matrix
        c = tf.matmul(a, b) #multiply them to get a 2x2 matrix
        print("Matrix multiplication result:", c)
    print("TensorFlow is using GPU:", tf.test.is_gpu_available())
else:
    print("No GPUs available")

Matrix multiplication result: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
TensorFlow is using GPU: True


In [14]:
gpus = tf.config.list_physical_devices('GPU')

# Set the GPU as the only visible device
tf.config.set_visible_devices(gpus[0], 'GPU')

# Run multiple iterations to avoid the first-run overhead
def benchmark(device_name, n_iterations=5, matrix_size=10000):
    print(f"Running on {device_name}:")
    times = []
    for _ in range(n_iterations):
        with tf.device(device_name):
            a = tf.random.uniform([matrix_size, matrix_size])
            b = tf.random.uniform([matrix_size, matrix_size])
            start = time.time()
            c = tf.matmul(a, b)
            tf.keras.backend.clear_session() #free GPU memory
            end = time.time()
            times.append(end - start)
            #print(f"Iteration time: {end - start:.4f} seconds")
    print(f"Average time on {device_name}: {sum(times) / n_iterations:.4f} seconds")
    print(f"Total time on {device_name}: {sum(times):.4f} seconds")
    

In [17]:
#compare your GPU to CPU given that prvious test pased
benchmark('/GPU:0',n_iterations=5 , matrix_size=10000)
benchmark('/CPU:0')

Running on /GPU:0:
Average time on /GPU:0: 0.0028 seconds
Total time on /GPU:0: 0.0140 seconds
Running on /CPU:0:
Average time on /CPU:0: 3.8280 seconds
Total time on /CPU:0: 19.1398 seconds
